In [12]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import googlemaps
import datetime as dt
import time

In [14]:
#set origin destination
destino = (-34.605537,-58.373022,)
origen = (-34.606652,-58.429120)

In [35]:
#census data
radios = gpd.read_file('../data/informacion_censal_por_radio_2010_GKBA/informacion_censal_por_radio_2010.shp')
radios = radios.loc[:,['CO_FRAC_RA','geometry']]
radios = radios.to_crs(epsg=4326)
radios['centroids'] = radios.geometry.centroid
radios.head()

,CO_FRAC_RA,geometry,centroids
0,1_1_1,POLYGON ((-58.37189352634645 -34.5919935640870...,POINT (-58.36825321588604 -34.58712576284138)
1,1_12_1,POLYGON ((-58.38592548750878 -34.6044398706025...,POINT (-58.38519884319477 -34.60504293372185)
2,1_12_10,POLYGON ((-58.37878669239328 -34.6052849003551...,POINT (-58.37684850487353 -34.60525008262525)
3,1_12_11,POLYGON ((-58.36732497447693 -34.6056209207810...,POINT (-58.37161228851214 -34.60570898563515)
4,1_12_2,POLYGON ((-58.38453585809192 -34.6043583533245...,POINT (-58.38378034377885 -34.60495283064369)


In [63]:
#query function
def queryGoogleAPI(origen):
    key = os.getenv('GOOGLEAPI')
    gmaps = googlemaps.Client(key=key)
    travel = gmaps.distance_matrix(origins = origen,
                               destinations = destino,
                               mode="transit", #driving
                               traffic_model="best_guess")
    tiempo = travel['rows'][0]['elements'][0]['duration']['value']
    dinero = travel['rows'][0]['elements'][0]['fare']['value']
    output = {'tiempo':tiempo,'dinero':dinero}
    return output


In [54]:
tiempos = []
dinero = []

googleLimit = 2400
lastQueryID = 0

while True:
    #check if the task is completed
    
    if len(tiempos) == radios.shape[0]:
        break
        
    #at 9 am
    elif ((dt.datetime.now().hour == 9) and (dt.datetime.now().minute == 0)):
        
        #start running the loop
        for i in range(lastQueryID,lastQueryID + googleLimit):
            
            #check if the task is completed
                        
            if i == radios.shape[0]:
                break
            
            #run googleQuery
            origen = (radios.centroids.loc[i].y,radios.centroids.loc[i].x)
            queryResult = queryGoogleAPI(origen)
            tiempos.append(queryResult['tiempo'])
            dinero.append(queryResult['dinero'])
            
        #save the last row of the query
        lastQueryID = i + 1
    
    time.sleep(60)


minuto par:  8
minuto par:  10


In [ ]:
radios['tiempos'] = tiempos
radios['dinero'] = dinero
radios.drop(['centroids'],axis=1,inplace=True)
ratios.to_file('../data/googleQuery')